# Segmenting and Clustering Neighborhoods in Toronto

Objective of this assignment is to explore, segment, and cluster the neighborhoods in the city of Toronto.

### Part 1: Scrape the wikipedia page for Toronto neighborhod data, clean & display thetop10 rows

Import the required packages like pandas, beautifulSoup and transform data into a dataframe from a html website

In [18]:
# Import the required packages

from bs4 import BeautifulSoup
import requests
import pandas as pd

In [19]:
# Download url data from internet

url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(url).text
CanadaPostaldata = BeautifulSoup(source, 'lxml')
# print(Canada_data)

In [20]:
# creat a new Dataframe

column_names = ['PostalCode','Borough','Neighborhood']
toronto = pd.DataFrame(columns = column_names)


In [29]:
#Scrape the toronto pincodes from html table 

content = CanadaPostaldata.find('div', class_ = 'mw-parser-output')
table = content.table.tbody
postcode = '' 
borough = '' 
neighborhood = ''
for tr in table.find_all('tr'):
    i = 0
    for td in tr.find_all('td'):
        if i == 0:
            postcode = td.text.strip('\n')
            i = i + 1
        elif i == 1:
            borough = td.text.strip('\n')
            i = i + 1
        elif i == 2: 
            neighborhood = td.text.strip('\n')
    toronto = toronto.append({'PostalCode': postcode,'Borough': borough,'Neighborhood': neighborhood},ignore_index=True)
    
toronto = toronto[toronto.Borough!='Not assigned']
toronto = toronto[toronto.Borough!= 0]
toronto.reset_index(drop = True, inplace = True)
i = 0
for i in range(0,toronto.shape[0]):
    if toronto.iloc[i][2] == 'Not assigned':
        toronto.iloc[i][2] = toronto.iloc[i][1]
        i = i+1

#Format the Neighborhood column
df = toronto.groupby(['PostalCode','Borough'])['Neighborhood'].apply(', '.join).reset_index()

# Drop NA & remove Not assigned values
df = df.dropna()
empty = 'Not assigned'
df = df[(df.PostalCode != empty ) & (df.Borough != empty) & (df.Neighborhood != empty)]

indexName = df[ df['PostalCode'] == '' ].index
df.drop(indexName , inplace=True)

In [31]:
# Group Neighborhood together
def group_neighborhood_list(grouped):    
    return ', '.join(sorted(grouped['Neighborhood'].tolist()))
                    
grouped_toronto = df.groupby(['PostalCode', 'Borough'])
df_totonto = grouped_toronto.apply(group_neighborhood_list).reset_index(name='Neighborhood')

df_totonto.head(11)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge, Malvern, Rouge, Malvern, Rouge..."
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek, Rouge ..."
2,M1E,Scarborough,"Guildwood, Morningside, West Hill, Guildwood, ..."
3,M1G,Scarborough,"Woburn, Woburn, Woburn, Woburn, Woburn, Woburn..."
4,M1H,Scarborough,"Cedarbrae, Cedarbrae, Cedarbrae, Cedarbrae, Ce..."
5,M1J,Scarborough,"Scarborough Village, Scarborough Village, Scar..."
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park, K..."
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge, Golden Mile, ..."
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village Wes..."
9,M1N,Scarborough,"Birch Cliff, Cliffside West, Birch Cliff, Clif..."


In [13]:
# Print the shape for the final dataframe
print('The shape of dataframe is',df_totonto.shape)

The shape of dataframe is (104, 3)
